<a href="https://colab.research.google.com/github/AlissonRP/LinearReg_to_nn/blob/master/mlg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression from scratch

Vamos aqui implementar um  dos modelos estatísticos mais básicos:
* Regressão Linear

Em uma perspectiva de redes neurais

## Regressão Linear
O modelo de regressão linear é simplesmente uma estrutura que prediz a média ($\mu_y$) de uma variável númerica $y$ utilizando combinação linear das covariáveis ($X_i$), ou seja:

$$
\mu_y = \sum_{i = 0}^{n}w_iX_i \quad \text{onde   \quad $X_0$ = 1}
$$




Na prática possuimos algumas observações $(x_i,y_i)$ e não conhecemos os $w_i$, portanto faz-se necessário algum método de estimação. O método mais conhecido é o método dos mínimos quadrados que minimiza a soma da diferença quadrática dos erros, ou seja:

$$
min_{w_i}\sum (Y - \hat{\mu}_y)²
$$

Mas aqui vamos tomar uma abordagem alternativa, vamos utilizar do gradiente descente como método de estimação dos $w_i$ e  tratar o modelo de regressão logística como um caso de uma rede neural.

## Deep Learning and PyTorch

Podemos examinar a regressão linear como um caso simples de uma rede neural onde só possuímos um neuronio e a função de ligação é a identidade. Assim os parâmetros $w_i$ serão encontrados por backpropagation na rede.

### Why simulated data?

Vamos simular os dados da nossa covariável $y$, pela seguinte função:
$$
y = Xw
$$

Onde os dados $X$ vem de distribuições Poisson com diferentes parâmetros. Vamos utilizar de dados simulados, pois sabemos **exatamente** o valor dos parâmetros, em outras palavras sabemos com exatidão a estrutura da função geradora dos dados, com isso podemos avaliar a aproximação da nn.

In [ ]:
import torch
import numpy as np



In [ ]:
np.random.seed(41)

X = torch.from_numpy(np.random.exponential(scale=15,size =(15,4)))
betas = torch.tensor([[1.,3.,15.,-2.]], dtype=torch.float64)
b0 = torch.tensor([[2.]], dtype=torch.float64)
y = X @ betas.t() + b0


Assim o valor dos parâmetros é especificado pelo vetor `beta` e `b0`

In [ ]:
betas

tensor([[ 1.,  3., 15., -2.]], dtype=torch.float64)

### Nothing new until now

Vamos aqui estabelecer alguns hiperparâmetros da rede :

* Loss: rmse := $\sqrt{\dfrac{1}{n}\sum_{i= n}^{n}(y - \hat{y})²}$
* Optimizer: Gradient descent := $w_i = w_{i-1} - \lambda \dfrac{d \text{Loss}}{dw_{i-1}}$
* $\lambda$ = 0.001
* Valores de inicialização dos pesos vem de uma distribuição normal padrão

In [ ]:
#https://www.analyticsvidhya.com/blog/2021/08/linear-regression-and-gradient-descent-in-pytorch/
# definindo a loss
def rmse(y,y_hat):
    return torch.sqrt(((torch.pow(y-y_hat,2))/torch.numel(y)).sum())



In [ ]:
w = torch.randn(1,int(X[1].numel()), requires_grad=True, dtype=torch.float64)

b =  torch.randn(1, requires_grad=True, dtype=torch.float64)

b

tensor([-1.7254], dtype=torch.float64, requires_grad=True)

In [ ]:
def LinearReg(inputs):
    y_t = inputs @ w.transpose(0,1)  + b
    return y_t
    

In [ ]:
#LinearReg(X)
loss = rmse(y,LinearReg(X))
loss.backward()
loss

tensor(505.0479, dtype=torch.float64, grad_fn=<SqrtBackward>)

In [ ]:

w.grad.zero_()
b.grad.zero_()

tensor([0.], dtype=torch.float64)

In [ ]:
learning_rate = 0.001
for i in range(1000):    
    y_hat = LinearReg(X)
    loss = rmse(y,y_hat)
    loss.backward()
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        w.grad.zero_()
        b.grad.zero_()
    if i == 50 or i==900:
        print(loss)

tensor(0.6461, dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.6461, dtype=torch.float64, grad_fn=<SqrtBackward>)


In [ ]:
w

tensor([[ 1.0184,  3.0285, 15.0179, -1.9772]], dtype=torch.float64,
       requires_grad=True)

In [ ]:
y

tensor([[ 261.2693],
        [  60.0724],
        [ 144.7189],
        [ 167.7342],
        [ 329.7891],
        [ 182.9445],
        [ 110.9456],
        [  79.9302],
        [1742.1217],
        [ 450.8701],
        [ 145.6723],
        [ 352.7420],
        [ 117.3411],
        [ 133.8173],
        [ 663.0963]], dtype=torch.float64)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(df.data,df.target)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(reg.predict(df.data),df.target)

2859.6903987680657